In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np 
from sklearn.metrics.pairwise import cosine_similarity

In [86]:
model = pd.read_parquet('games_reviews_items.parquet')

In [100]:
genrestitle = model.groupby('title')['genres'].agg(', '.join).reset_index()

In [140]:
genrestitle.tail(2)

,title,genres
2819,[the sequence],Indie
2820,Астролорды: облако оорта,"Massively multiplayer, Free to play, Indie, St..."


In [115]:
wf = CountVectorizer(max_features = 8000, stop_words="english")

In [116]:
wf.fit_transform(genrestitle["genres"]).toarray().shape

(2821, 29)

In [117]:
vectors = wf.fit_transform(genrestitle["genres"]).toarray()

In [118]:
vectors

array([[0, 1, 1, ..., 0, 0, 0],
       [0, 2, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 2, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [158]:
similarity = cosine_similarity(vectors)

In [159]:
similarity[0]

array([1.        , 0.70710678, 0.35355339, ..., 0.70710678, 0.5       ,
       0.20412415])

In [160]:
sorted(list(enumerate(similarity[0])), reverse=True, key=lambda x:x[1])[1:6]

[(142, 1.0), (225, 1.0), (239, 1.0), (300, 1.0), (329, 1.0)]

FUNCION MODELO CON EL DATAFRAME AL QUE SE LE APLICO LA SIMILITUD DEL COSENO

In [173]:
def obtener_recomendaciones(game, genrestitle, similarity, n=5):
    
    if game not in genrestitle['title'].values:
        return []  # Retorna una lista vacía si el juego no se encuentra
    
    
    indice_juego = genrestitle[genrestitle["title"] == game].index[0]
    distances = similarity[indice_juego]   
    top_indices = sorted(range(len(distances)), key=lambda i: distances[i], reverse=True)[1:n+1]  
    recommended_titles = [genrestitle.iloc[i]['title'] for i in top_indices]
    
    return recommended_titles


In [174]:
obtener_recomendaciones("Zero g arena", genrestitle, similarity)

['Bunker punks', 'Gamma bros 1.5', 'Garbage day', 'Gunjitsu', 'Noct']

In [175]:
obtener_recomendaciones("Zoombinis", genrestitle, similarity)

['Clannad',
 "Dark parables: the exiled prince collector's edition",
 'Enigmatis 2: the mists of ravenwood',
 'Freddi fish 3: the case of the stolen conch shell',
 'Freddi fish 5: the case of the creature of coral cove']

In [176]:
obtener_recomendaciones("Астролорды: облако оорта", genrestitle, similarity)

['Faeria',
 'Infinity wars: animated trading card game',
 'Saga',
 'Spellweaver',
 'Астролорды: облако оорта']

In [177]:
obtener_recomendaciones("0rbitalis", genrestitle, similarity)

['Euro truck simulator 2',
 "Five nights at freddy's",
 "Garry's mod",
 'Hacknet',
 'Huniecam studio']

In [178]:
obtener_recomendaciones("10,000,000", genrestitle, similarity)

['Starbound',
 'Castle crashers®',
 'Forward to the sky',
 'Tap heroes',
 'Time ramesside (a new reckoning)']

In [179]:
obtener_recomendaciones("10,0", genrestitle, similarity)

[]

GENERO LA COLUMNA RECOMMENDED PARA ANEXARLA AL DATAFRAME CON LA INTENCION DE ELIMINAR POSTERIORMENTE TODAS LAS COLUMNAS Y QUEDARME CON UN DATAFRAME RESULTADO QUE CONTENGA ITEM_ID Y RECOMMENDED

In [211]:
def obtener_recomendaciones_para_fila(row):
    return obtener_recomendaciones(row['title'], genrestitle, similarity)

genrestitle['recommended'] = genrestitle.apply(obtener_recomendaciones_para_fila, axis=1)

In [212]:
genrestitle

,title,genres,recommended
0,! that bastard is trying to steal our gold !,"Action, Adventure, Casual, Indie",[Aperture tag: the paint gun testing initiativ...
1,//n.p.p.d. rush//- the milk of ultraviolet,"Action, Action, Indie, Indie","[404sight, And yet it moves, Assault android c..."
2,0rbitalis,"Simulation, Indie","[Euro truck simulator 2, Five nights at freddy..."
3,"10,000,000","Action, Action, Casual, Casual, Casual, Indie,...","[Starbound, Castle crashers®, Forward to the s..."
4,100% orange juice,"Indie, Indie, Strategy, Indie, Indie, Indie, I...","[Ai war: fleet command, Defense grid: the awak..."
...,...,...,...
2816,Zombie zoeds,"Adventure, Indie, Action, Casual",[Aperture tag: the paint gun testing initiativ...
2817,Zoombinis,"Adventure, Adventure, Adventure, Casual, Casua...","[Clannad, Dark parables: the exiled prince col..."
2818,Zuma's revenge!,"Action, Action, Casual, Casual","[Flight control hd, Fury of the gods, Heli her..."
2819,[the sequence],Indie,"[Adventures of shuggy, Analogue: a hate story,..."


In [213]:
result = genrestitle.merge(model[['title', 'item_id']], on='title', how='inner')
result

,title,genres,recommended,item_id
0,! that bastard is trying to steal our gold !,"Action, Adventure, Casual, Indie",[Aperture tag: the paint gun testing initiativ...,449940
1,! that bastard is trying to steal our gold !,"Action, Adventure, Casual, Indie",[Aperture tag: the paint gun testing initiativ...,449940
2,! that bastard is trying to steal our gold !,"Action, Adventure, Casual, Indie",[Aperture tag: the paint gun testing initiativ...,449940
3,! that bastard is trying to steal our gold !,"Action, Adventure, Casual, Indie",[Aperture tag: the paint gun testing initiativ...,449940
4,//n.p.p.d. rush//- the milk of ultraviolet,"Action, Action, Indie, Indie","[404sight, And yet it moves, Assault android c...",270090
...,...,...,...,...
113958,[the sequence],Indie,"[Adventures of shuggy, Analogue: a hate story,...",454320
113959,Астролорды: облако оорта,"Massively multiplayer, Free to play, Indie, St...","[Faeria, Infinity wars: animated trading card ...",385530
113960,Астролорды: облако оорта,"Massively multiplayer, Free to play, Indie, St...","[Faeria, Infinity wars: animated trading card ...",385530
113961,Астролорды: облако оорта,"Massively multiplayer, Free to play, Indie, St...","[Faeria, Infinity wars: animated trading card ...",385530


In [214]:
result = result.drop(['title'], axis=1)
result = result.drop(['genres'], axis=1)

In [243]:
result.tail(2)

,recommended,item_id
113961,"[Faeria, Infinity wars: animated trading card ...",385530
113962,"[Faeria, Infinity wars: animated trading card ...",385530


In [218]:
result['item_id'] = result['item_id'].astype('int64')

In [245]:
result.to_parquet('result.parquet', engine = 'pyarrow')

In [244]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113963 entries, 0 to 113962
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   recommended  113963 non-null  object
 1   item_id      113963 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.7+ MB


FUNCION QUE FILTRA FILA DE LA COLUMNA ITEM_ID SEGUN EL VALOR QUE SE INGRESE Y TRAE COMO RESULTADO EL VALOR CORRESPONDIENTE DE LA COLUMNA RECOMMENDED

In [260]:
def obtener_recomendaciones_por_item_id(item: int):
    recomendaciones = result[result['item_id'] == item]['recommended'].iloc[0]
    
    if len(recomendaciones) > 0:
        recomendaciones_dict = {i + 1: juego for i, juego in enumerate(recomendaciones)}
        return recomendaciones_dict
    else:
        
        error_data = {'error': 'No se encontraron recomendaciones para el ID proporcionado'}
        return JSONResponse(content=error_data, status_code=404)


In [261]:
obtener_recomendaciones_por_item_id(10)

{1: 'Alien breed 2: assault',
 2: 'Alien breed: impact',
 3: 'Alien rage - unlimited',
 4: 'Alien: isolation',
 5: 'Aliens versus predator classic 2000'}

In [262]:
obtener_recomendaciones_por_item_id(449940)

{1: 'Aperture tag: the paint gun testing initiative',
 2: 'Battleblock theater®',
 3: 'Beatbuddy: tale of the guardians',
 4: 'Bloody trapland',
 5: 'Broforce'}

In [263]:
obtener_recomendaciones_por_item_id(385530)

{1: 'Faeria',
 2: 'Infinity wars: animated trading card game',
 3: 'Saga',
 4: 'Spellweaver',
 5: 'Астролорды: облако оорта'}

In [264]:
obtener_recomendaciones_por_item_id(270090)

{1: '404sight',
 2: 'And yet it moves',
 3: 'Assault android cactus',
 4: 'Astebreed: definitive edition',
 5: 'Blade symphony'}

In [265]:
obtener_recomendaciones_por_item_id(220820)

{1: 'Carmageddon max pack',
 2: 'O3dx',
 3: 'Obliteracers',
 4: 'Speed kills',
 5: 'T.e.c. 3001'}

ESTA DAFRAME DE NOMBRE RESULT Y SU CORRESPONDIENTE ARCHIVO PARQUET SERAN LOS USADOS EN EL DESPLIEGUE DE LA API

In [2]:
def obtener(item_id):
    fila = result[result['item_id'] == item_id]['recommended'].iloc[0]
    # Verificar si la lista de recomendaciones no está vacía
    if len(fila) > 0:
        fila_dict = {i + 1: juego for i, juego in enumerate(fila)}
        return fila_dict
    else:
        # Si la lista de recomendaciones está vacía, devolver un diccionario vacío
        return {}

In [3]:
result = pd.read_parquet('result.parquet')

In [5]:
result.head()

,recommended,item_id
0,[Aperture tag: the paint gun testing initiativ...,449940
1,[Aperture tag: the paint gun testing initiativ...,449940
2,[Aperture tag: the paint gun testing initiativ...,449940
3,[Aperture tag: the paint gun testing initiativ...,449940
4,"[404sight, And yet it moves, Assault android c...",270090


In [9]:
obtener(449940)

{1: 'Aperture tag: the paint gun testing initiative',
 2: 'Battleblock theater®',
 3: 'Beatbuddy: tale of the guardians',
 4: 'Bloody trapland',
 5: 'Broforce'}

In [10]:
obtener(270090)

{1: '404sight',
 2: 'And yet it moves',
 3: 'Assault android cactus',
 4: 'Astebreed: definitive edition',
 5: 'Blade symphony'}